# Mini Batch and Data Load

우리가 훈련시키는 데이터는 매우 큰 경우가 많고 그에 따라서 많은 계산량을 필요로 한다.

그렇기 때문에 우리는 전체 데이터를 한번에 학습시키는 것이 아니라 일정 단위로 나누어서 학습을 하는 방법을 선택하게 된다.

미니 배치(Mini Batch)는 앞서 말한 방식에서 데이터를 일정 단위로 나눌때 사용되는 단위이다.

전체 데이터를 미니 배치로 나눈다음에 이 미니 배치 수 만큼 학습을 완료하면 1 Epoch가 끝이 난 것이다.

미니 배치의 크기를 어떻게 하는가에 대한 문제가 있는데 미니 배치의 크기를 배치 크기(Batch Size)라고 한다.

전체 데이터를 미니 배치로 나누어 1개의 미니배치를 학습하는 것이 이터레이션(Iteration)이라고 한다.

예를들어 2000개의 데이터를 배치 크기 200으로 나누면 총 10개의 미니 배치로 나누어진다.

따라서 10번의 이터레이션이 진행될 것이다. 즉, 가중치와 편향이 10번 업데이트가 이루어 진다는 것을 의미한다. 

미니 배치로 데이터를 나누어 학습하는 것을 '미니 배치 경사 하강법'이라고 하고, 전체 데이터를 한 번에 하는 것을 '배치 경사 하강법'이라고 부른다.

## 데이터 로드하기

PyTorch에서는 데이터를 로드하기 위해서 Dataset과 DataLoader를 제공한다.

기본적으로는 Dataset을 정의하고 DataLoader에 전달하는 것이다.

이를 이용해서 미니 배치 학습, 데이터 셔플, 병렬 처리를 수행할 수 있다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [2]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

In [3]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

DataLoader를 이용할때는 Dataset과 batch size를 정의해주어야한다.

보통 batch size는 2의 배수를 상용한다.

shuffle은 Epoch를 진행할때마다 데이터의 순서를 Random하게 섞는 것을 의미한다.

똑같은 데이터여도 데이터의 배치를 Random하게 섞으면 더 잘 학습이 가능하다.

In [4]:
model = nn.Linear(3,1) #3가지 값을 입력 받아서 1개의 값을 출력하는 다중 선형 회귀 모델

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) #SGD를 이용하여, Learning rate는 1e-5

In [5]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 34233.855469
Epoch    0/20 Batch 2/3 Cost: 13817.115234
Epoch    0/20 Batch 3/3 Cost: 5960.549805
Epoch    1/20 Batch 1/3 Cost: 730.721313
Epoch    1/20 Batch 2/3 Cost: 437.575684
Epoch    1/20 Batch 3/3 Cost: 110.303520
Epoch    2/20 Batch 1/3 Cost: 5.684276
Epoch    2/20 Batch 2/3 Cost: 25.150545
Epoch    2/20 Batch 3/3 Cost: 20.171661
Epoch    3/20 Batch 1/3 Cost: 4.058476
Epoch    3/20 Batch 2/3 Cost: 11.481176
Epoch    3/20 Batch 3/3 Cost: 8.568769
Epoch    4/20 Batch 1/3 Cost: 8.175518
Epoch    4/20 Batch 2/3 Cost: 4.682809
Epoch    4/20 Batch 3/3 Cost: 11.290629
Epoch    5/20 Batch 1/3 Cost: 5.878546
Epoch    5/20 Batch 2/3 Cost: 8.409691
Epoch    5/20 Batch 3/3 Cost: 8.666419
Epoch    6/20 Batch 1/3 Cost: 0.360738
Epoch    6/20 Batch 2/3 Cost: 18.866890
Epoch    6/20 Batch 3/3 Cost: 10.444901
Epoch    7/20 Batch 1/3 Cost: 9.642101
Epoch    7/20 Batch 2/3 Cost: 7.243816
Epoch    7/20 Batch 3/3 Cost: 2.639088
Epoch    8/20 Batch 1/3 Cost: 7.982546
Ep

In [6]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[155.3204]], grad_fn=<AddmmBackward0>)


## Custom Dataset

우리는 위에서 Dataset을 정의한 다음 DataLoader를 이용하여 Mini Batch 방법을 수행하였다.

이번에는 직접 Dataset을 만들어 사용해 볼 것이다.

직접 Dataset을 만들기 위해서는 Class를 사용해야하는데, 우리는 3가지 define을 사용할 것이다.

'__init__', '__len__'과 '__getitem__'이다.

In [12]:
# Dataset 상속
class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [14]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [15]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [16]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 10097.804688
Epoch    0/20 Batch 2/3 Cost: 2283.451416
Epoch    0/20 Batch 3/3 Cost: 507.059875
Epoch    1/20 Batch 1/3 Cost: 297.864960
Epoch    1/20 Batch 2/3 Cost: 141.844543
Epoch    1/20 Batch 3/3 Cost: 20.542185
Epoch    2/20 Batch 1/3 Cost: 15.037828
Epoch    2/20 Batch 2/3 Cost: 8.075519
Epoch    2/20 Batch 3/3 Cost: 0.063480
Epoch    3/20 Batch 1/3 Cost: 5.601187
Epoch    3/20 Batch 2/3 Cost: 2.735129
Epoch    3/20 Batch 3/3 Cost: 1.664595
Epoch    4/20 Batch 1/3 Cost: 7.428864
Epoch    4/20 Batch 2/3 Cost: 3.328489
Epoch    4/20 Batch 3/3 Cost: 0.627107
Epoch    5/20 Batch 1/3 Cost: 3.193416
Epoch    5/20 Batch 2/3 Cost: 1.095299
Epoch    5/20 Batch 3/3 Cost: 9.350142
Epoch    6/20 Batch 1/3 Cost: 3.171443
Epoch    6/20 Batch 2/3 Cost: 5.904304
Epoch    6/20 Batch 3/3 Cost: 1.769514
Epoch    7/20 Batch 1/3 Cost: 2.959316
Epoch    7/20 Batch 2/3 Cost: 4.274165
Epoch    7/20 Batch 3/3 Cost: 1.429538
Epoch    8/20 Batch 1/3 Cost: 6.805875
Epoch    8

In [17]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[153.0042]], grad_fn=<AddmmBackward0>)
